In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
df = pd.read_csv('../../data/swat/swat_ieee754.csv')

In [3]:
inputs = df.iloc[:, :-1].values
labels = (df.iloc[:, -1]).values

In [4]:
import numpy as np

def find_segments_as_dict(labels):
    """
    找出每段连续相同标签的开始和结束索引，并将结果以字典形式返回，
    键为（开始索引，结束索引），值为对应的标签。

    参数：
        labels (np.ndarray): 标签数组（一维数组）。
    
    返回：
        dict: 每段的开始和结束索引及对应标签，形式为 {(start1, end1): label1, (start2, end2): label2, ...}。
    """
    # 找出标签变化的位置
    changes = np.diff(labels) != 0
    # 开始索引是变化点的下一个位置，补上第一个位置
    starts = np.r_[0, np.where(changes)[0] + 1]
    # 结束索引是变化点本身，补上最后一个位置
    ends = np.r_[np.where(changes)[0], len(labels) - 1]
    
    # 返回以 (start, end) 为键，标签为值的字典
    return {(start, end): labels[start] for start, end in zip(starts, ends)}



In [5]:
attack_index = np.where(labels != 0)[0]
select_normal_index = np.r_[734:1734, 2697:3046, 3900:4900, 5458:6458, 6849:7232, 7432:7684, 10384:11384, 14360:15360, 89668:90668, 91139:92139, 92517:93423, 102091:103091, 114821:115821, 116080:116122, 116515:116998, 131893:132893, 141926:142926, 171267:172267, 172589:172891, 197272:198272, 226827:227827, 228520:229520, 279059:280059, 301652:302652, 303020:304020]
index = np.concatenate((select_normal_index, attack_index), axis=0)
inputs, labels = inputs[index], labels[index]
